In [21]:
import simpy
import random
import numpy as np
from scipy import stats

In [22]:
class FiFoServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.Resource(env, capacity=num_servers)

class SJFServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.PriorityResource(env, capacity=num_servers)

def task(name, env, servers, processing_time, waiting_times, print_tasks):

    # A task arrives at the server
    start_time = env.now
    if print_tasks:
        print(f'Task {name} arriving at {start_time}')

    # Request task to server
    with servers.server.request() as req:

        # Once there is availability at the server, initiate task
        yield req
        end_time = env.now
        if print_tasks:
            print(f'Server starts processing {name} at {end_time}')

        yield env.timeout(processing_time)

        if print_tasks:
            print(f'Server done processing {name} at {env.now}')

        waiting_times.append(end_time - start_time)


def task_generator(env, servers, N, lambda_, waiting_times, print_tasks=False):

    # Generate N tasks
    for i in range(N):
        
        # Calculate time at server by Poisson process, not sure if necessary
        processing_time = max(0, int(random.normalvariate(1 / servers.mu, 1)))

        # Create task
        env.process(task(i, env, servers, processing_time, waiting_times, print_tasks))

        # Calculate time until next task by Poisson process, not sure if necessary
        next_arrival = max(0, int(random.normalvariate(1 / lambda_, 1)))
        yield env.timeout(next_arrival)


In [23]:
def run_simulation(num_servers, mu, lambda_, N, tmax, scheduler):
    env = simpy.Environment()
    if scheduler.lower() == 'fifo':
        servers = FiFoServers(env, mu, num_servers)
    elif scheduler.lower() == 'sjf':
        servers = SJFServers(env, mu, num_servers)

    waiting_times = []

    env.process(task_generator(env, servers, N, lambda_ * num_servers, waiting_times))
    env.run(tmax)

    waiting_times_array = np.array(waiting_times)
    average_waiting_time = np.mean(waiting_times_array)
    std_dev_waiting_time = np.std(waiting_times_array)
    
    # Calculate confidence interval for the mean
    confidence_level = 0.95
    lamb = stats.t.ppf((1 + confidence_level) / 2, len(waiting_times_array) - 1)
    n = len(waiting_times_array)

    margin_of_error = lamb * (std_dev_waiting_time / np.sqrt(n))
    confidence_interval = (average_waiting_time - margin_of_error, average_waiting_time + margin_of_error)

    print(f"{num_servers} servers:")
    print(f"Average Waiting Time: {average_waiting_time}")
    print(f"Standard Deviation of Waiting Time: {std_dev_waiting_time}")
    print(f"Confidence Interval (95%): {confidence_interval}\n")


mu = 0.2
lambda_ = 0.15
N = 100000
tmax = 200000
num_servers_list = [1, 2, 4]
scheduler = 'fifo'

for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 0.11283206318842369
Standard Deviation of Waiting Time: 0.4106798754549174
Confidence Interval (95%): (0.10836088975198382, 0.11730323662486355)

2 servers:
Average Waiting Time: 0.4726494145765461
Standard Deviation of Waiting Time: 0.9963025675462573
Confidence Interval (95%): (0.46528400405207787, 0.4800148251010143)

4 servers:
Average Waiting Time: 3.93418
Standard Deviation of Waiting Time: 4.480799898187823
Confidence Interval (95%): (3.9064078886763984, 3.9619521113236016)



In [24]:
scheduler = 'sjf'

for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 0.1129320987654321
Standard Deviation of Waiting Time: 0.41536022780980425
Confidence Interval (95%): (0.10840920153698085, 0.11745499599388336)

2 servers:
Average Waiting Time: 0.5013494609221552
Standard Deviation of Waiting Time: 1.041509580030081
Confidence Interval (95%): (0.4936758944675226, 0.5090230273767877)

4 servers:
Average Waiting Time: 4.18806
Standard Deviation of Waiting Time: 5.0252117802536445
Confidence Interval (95%): (4.156913610034601, 4.219206389965399)

